### Introduction

Welcome to Priceless Wheels! In this project, our goal is to build a model that can accurately predict the price of a used vehicle based on various factors such as make, model, year, mileage, and condition. The automobile industry is one of the largest and most competitive industries in the world, with millions of vehicles being sold each year. The price of a vehicle can have a significant impact on a consumer's purchasing decision and it is important for both buyers and sellers to have an understanding of the market value of a vehicle. By using machine learning algorithms and data analysis, we aim to provide a reliable and robust model that can assist in determining the fair market value of a vehicle. Join us on this exciting journey as we delve into the world of vehicle price prediction.

### About the data

This data is scrapped from https://www.cardekho.com/. This data is meant for research and academic purposes only and is **not meant for commercial use**. This dataset contains about 38000 (thirty eight thousand) used cars listed in CarDekho in India. Download the data from https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv.

---

#### Importing the data and libraries

Let's start by importing the necessary libraries and the data.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt # 
import seaborn as sns # seaborn for nice looking plots
sns.set() # setting seaborn default for plots

# removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/used-cars-dataset-cardekho/cars_details.csv
/kaggle/input/used-cars-dataset-cardekho/cars_overview.csv
/kaggle/input/used-cars-dataset-cardekho/cars_details_merges.csv
/kaggle/input/used-cars-dataset-cardekho/feature_dictionary.csv


Next, we will import the data and preview the first 5 rows

- Note: The ```usedCarSkuId``` is a unique identifier for each car. We would be using this column as the index of the dataframe
- Note: Since we are using github lfs for storing the CSV files, the link mentioned in the notebook will expire in some time. Please visit this [link](https://github.com/chats-bug/Priceless-Wheels/blob/data-cleaning/data/raw/cardekho_cars_2023_03_19_16_44_14.csv) to get the data

In [2]:
# The data can be found on kaggle link: https://www.kaggle.com/datasets/sukritchatterjee/used-cars-dataset-cardekho

file_path = "/kaggle/input/used-cars-dataset-cardekho/cars_details_merges.csv"
df = pd.read_csv(file_path, index_col="usedCarSkuId")

# sanity check
df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (25,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,position,loc,myear,bt,tt,ft,km,ip,pi,images,...,owner_type,price_segment_new,template_name_new,page_template,template_Type_new,experiment,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,1,Gomti Nagar,2016,Hatchback,Manual,CNG,"69,162",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,2,Borivali West,2015,Hatchback,Manual,CNG,"45,864",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,3,JASOLA,2015,Sedan,Manual,CNG,"81,506",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,4,jasola,2013,Hatchback,Manual,CNG,"1,15,893",0,NaN,[{'img': ''}],...,second,2lakh-5lakh,used cardetail v2/corporate/13,Used Car > Detail Page,used,control,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,5,mumbai g.p.o.,2022,MUV,Manual,CNG,"18,900",0,https://images10.gaadi.com/usedcar_image/origi...,[{'img': 'https://images10.gaadi.com/usedcar_i...,...,first,10+lakh,used cardetail v2,Used Car > Detail Page,used,control,NaN,12.0+-.03,NaN,NaN


The data has imported with some mixed data types. We will need to convert the data types as we explore the data.

In [3]:
# checking the shape of the dataset
shape = df.shape

print(f"The shape of the dataframe is {shape[0]} rows and {shape[1]} columns")

The shape of the dataframe is 37814 rows and 139 columns


As we can see, there are 37,814 rows and 139 columns in the dataset.

Let's look at the data types of the columns.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37814 entries, 7111bf25-97af-47f9-867b-40879190d800 to a96fbcd7-c183-4829-ae97-b2581afe4bac
Columns: 139 entries, position to Ground Clearance Unladen
dtypes: bool(3), float64(10), int64(28), object(98)
memory usage: 39.6+ MB


Let's take a look at the feature dictionary provided with the dataset.

In [5]:
feature_dictionary = pd.read_csv("/kaggle/input/used-cars-dataset-cardekho/feature_dictionary.csv")
feature_dictionary

,Feature,Type,Sample,Missing Values,Unique Values,Description
0,position,int64,1,0,20,Position of the car in the list
1,loc,object,Gomti Nagar,5851,511,Location of the car
2,myear,int64,2016,0,34,Manufacturing year of the car
3,bt,object,Hatchback,19,11,Body type of the car
4,tt,object,Manual,0,2,Transmission type of the car
...,...,...,...,...,...,...
135,experiment,object,control,0,1,control
136,Fuel Suppy System,object,NaN,5502,99,"Type of fuel supply system (Carburetor, Fuel I..."
137,Compression Ratio,object,NaN,27642,100,Compression ratio of the engine
138,Alloy Wheel Size,object,NaN,13146,18,Size of the alloy wheels in inches


Now comes the difficult task. Go through the feature dictionary and remove the unwanted columns - features which might be repeated but not totally apparent. Note down the columns which makes sense to keep and are not repeating and delete the rest.

In [6]:
columns_to_keep = [
    "loc",
    "myear",
    "bt",
    "tt",
    "ft"
    "km",
    "ip",
    "images",
    "imgCount",
    "threesixty",
    "dvn",
    "oem",
    "model",
    "variantName",
    "city_x",
    "pu",
    "discountValue",
    "utype",
    "carType", 
    "top_features",
    "comfort_features",
    "interior_features",
    "exterior_features",
    "safety_features",
    "Color",
    "Engine Type",
    "Max Power",
    "Max Torque",
    "No of Cylinder",
    "Values per Cylinder",
    "Value Configuration",
    "BoreX Stroke",
    "Turbo Charger",
    "Super Charger",
    "Length",
    "Width",
    "Height",
    "Wheel Base",
    "Front Tread",
    "Rear Tread",
    "Kerb Weight",
    "Gross Weight",
    "Gear Box",
    "Drive Type",
    "Seating Capacity",
    "Steering Type",
    "Turning Radius",
    "Front Brake Type",
    "Rear Brake Type",
    "Top Speed",
    "Acceleration",
    "Tyre Type",
    "No Door Numbers",
    "Cargo Volumn",
    "model_type_new",
    "state",
    "owner_type",
    "exterior_color",
    "Fuel Suppy System",
    "Compression Ratio",
    "Alloy Wheel Size",
    "Ground Clearance Unladen",
]

df.drop([x for x in df.columns if x not in columns_to_keep], axis=1, inplace=True)
print(f"After dropping some unnecessary columns, the dataset now has {df.shape[1]} columns. These columns are hand picked and will be be further analyzed.")

After dropping some unnecessary columns, the dataset now has 60 columns. These columns are hand picked and will be be further analyzed.


---

### Data cleaning and wrangling

* Dropping duplicate rows
* Fixing the values and data types of the columns
* Checking for multicollinearity and determining how to handle it
* Dropping irrelevant columns for the model
* Saving the cleaned data for the next part of the project

---

### Duplicates

We know that there should be no duplicates in the data. They are checked for and removed at the time of data collection. *(Trust me, I collected it myself 😉 )*

In [7]:
# checking for duplicate rows
duplucate_rows = df.duplicated().sum()

print(f"The number of duplicate rows are {duplucate_rows}.")

The number of duplicate rows are 1.


Somehow, one duplicate row sneaked in (that's awkward 🫣)! Let's check the rows.

In [8]:
# setting the duplicated index
duplicate_index = df.duplicated(keep=False)

# calling the duplicated index in a dataframe
df.loc[duplicate_index, :].sort_index()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
aa39e640-6183-4379-a517-9f5b2458b2a5,NaN,2014,Hatchback,Manual,0,[{'img': ''}],0,False,Maruti Swift VDI BSIV,Maruti,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN
ce8b30d8-c438-4a2f-bce1-5c1887a95495,NaN,2014,Hatchback,Manual,0,[{'img': ''}],0,False,Maruti Swift VDI BSIV,Maruti,...,5.000,204-liters,used,uttar pradesh,Silver,second,CRDI,17.6:1,NaN,NaN


As we can see, the rows are exactly the same. This means we can safely drop the row. Since the we indexed by the usedCarSkuId, we need to drop one of them by their index.

In [9]:
# making a copy with the duplicated rows dropped
df2 = df.drop_duplicates().copy()

# checking for duplicate rows in the new dataframe
dup = df2.duplicated().sum()

print(f"The number of duplicate rows are {dup}.")

The number of duplicate rows are 0.


In [10]:
# sanity check
df2.head()

,loc,myear,bt,tt,ip,images,imgCount,threesixty,dvn,oem,...,No Door Numbers,Cargo Volumn,model_type_new,state,exterior_color,owner_type,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen
usedCarSkuId,,,,,,,,,,,,,,,,,,,,,
7111bf25-97af-47f9-867b-40879190d800,Gomti Nagar,2016,Hatchback,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,Maruti Wagon R LXI CNG,Maruti,...,5.000,180-liters,used,uttar pradesh,Silver,first,NaN,NaN,NaN,NaN
c309efc1-efaf-4f82-81ad-dcb38eb36665,Borivali West,2015,Hatchback,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,Maruti Celerio Green VXI,Maruti,...,5.000,235-litres,used,maharashtra,Grey,first,Intelligent-Gas Port Injection,11.0:1,NaN,NaN
7609f710-0c97-4f00-9a47-9b9284b62d3a,JASOLA,2015,Sedan,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,False,Honda Amaze S Plus i-VTEC,Honda,...,4.000,400-litres,used,delhi,Silver,second,NaN,NaN,NaN,NaN
278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,Hatchback,Manual,0,[{'img': ''}],0,False,Maruti Wagon R LXI CNG,Maruti,...,4.000,NaN,used,delhi,Silver,second,MPFI,NaN,13,NaN
b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,MUV,Manual,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,False,Maruti Ertiga VXI CNG,Maruti,...,5.000,NaN,used,maharashtra,White,first,NaN,12.0+-.03,NaN,NaN


Great! Now that we have removed the duplicate rows, let's look at missing values

---

### Fixing the data types

Let's look at the data types of the columns and in the process we can check if we need to clean any them to make them more useful, such as converting them to a numeric type.

*When we do feature engineering we will further modify these columns and might also drop a few of them*

We already know about the current data types of the features from the ```feature_dictionary```.

In [11]:
# Filter the dataframe to only include the columns_to_keep
new_feature_dict = feature_dictionary.copy()

# Drop the rows which Feature is not in the columns_to_keep list
row_index = [i for i, x in enumerate(new_feature_dict['Feature']) if x not in columns_to_keep]
new_feature_dict.drop(row_index, inplace=True)

new_feature_dict

,Feature,Type,Sample,Missing Values,Unique Values,Description
1,loc,object,Gomti Nagar,5851,511,Location of the car
2,myear,int64,2016,0,34,Manufacturing year of the car
3,bt,object,Hatchback,19,11,Body type of the car
4,tt,object,Manual,0,2,Transmission type of the car
7,ip,int64,0,0,2,Insurance premium of the car
9,images,object,[{'img': 'https://images10.gaadi.com/usedcar_i...,0,37135,Number of images of the car
10,imgCount,int64,15,0,54,Count of images of the car
11,threesixty,bool,False,0,2,Whether 360 degree view of car is available or...
12,dvn,object,Maruti Wagon R LXI CNG,0,4159,Dealer verification status of the car
13,oem,object,Maruti,0,46,Original equipment manufacturer of the car


We know that if a column has the ```object``` type, it means it could have mixed types as well as strings. Let's try to convert them to a numerical type wherver we encouter something like Length or '12mm'.

In [12]:
# A helper function to print all the types in a column for a dataframe
def print_samples_from_every_type(df: pd.DataFrame, col: str):
    # Get all the types in the column.
    types = df[col].apply(type).unique()
    for t in types:
      # Print 10 samples from each type.
        print(f"Samples for type {t}: ")
        print(df[df[col].apply(type) == t].sample(10)[col])
        print()

In [13]:
# Make a copy of the original dataframe so that we can roll back to it anytime.
cars_df = df2.copy()

---

Let's list down all the columns which need the standard string treatment: 
1. Convert column to be a string type
2. Strip the whitespace 
3. Make it lowercase

In [14]:
columns_to_keep_str = [
    'loc',
    'bt',
    'tt',
    'images',
    'dvn',
    'oem',
    'model',
    'variantName',
    'city_x',
    # 'pu': 3,70,000 -> 370000
    'utype',
    'carType',
    'top_features',
    'comfort_features',
    'interior_features',
    'exterior_features',
    'safety_features',
    'Color',
    'Engine Type',
    # Max Power : 33.54bhp@4000 rpm -> 2 columns 33.54, 4000
    # Max Torque : 40.2Nm@3500 rpm -> 2 columns 40.2, 3500
    # No of Cylinder : float -> int,
    # Values per Cylinder : float -> int, !! Fix the name -> Valve per Cylinder
    'Value Configuration', # !! Fix the name -> Valve Configuration
    # BoreX Stroke : 69 x 72 mm -> Bore: 69, Stroke: 72
    'Turbo Charger', # Convert to boolean
    'Super Charger', # Convert to boolean
    # Length : 3599mm -> 3599
    # Width : 1495mm -> 1495
    # Height : 1700mm -> 1700
    # Wheel Base : 2400mm -> 2400
    # Front Tread : 1295mm -> 1295
    # Rear Tread : 1295mm -> 1295
    # Kerb Weight : 960kg -> 960
    # Gross Weight : 1350kg -> 1350
    'Gear Box', # might need some additional cleaning
    'Drive Type', # might need some additional cleaning
    # Seating Capacity : float -> int
    'Steering Type', # might need some additional cleaning
    # Turning Radius : 4.6 metres -> 4.6
    'Front Brake Type', # might need some additional cleaning
    'Rear Brake Type', # might need some additional cleaning
    # Top Speed	: 137 kmph -> 137
    # Acceleration : 13.5 seconds -> 13.5
    'Tyre Type', # might need some additional cleaning
    # No Door Numbers : float -> int !! fix name
    # 'Cargo Volumn' : 300 litres -> 300, !! fix name
    'model_type_new', # !! fix name
    'state',
    'exterior_color',
    'owner_type', # might need some additional cleaning
    'Fuel Suppy System', # might need some additional cleaning
    # Compression Ratio : 10.0:1 -> 10.0
    # Alloy Wheel Size	: convert to float
    # Ground Clearance Unladen : 170mm -> 170
]

In [15]:
# Apply the changes
temp_arr = []

for col in columns_to_keep_str:
    cars_df[col] = cars_df[col].astype(str).str.strip().str.lower()
    
    # Print a nice sample
    while (True):
        found = 0
        vals = [v for v in cars_df[col].sample(100)]
        for val in vals:
            if val is not None:
                found = 1
                vals = val
                break
        
        if found == 1:
            temp_arr.append({ 
                "Feature" : col, 
                "Type": f'{cars_df[col].dtype}', 
                "Unique": cars_df[col].nunique(), 
                "Sample": vals
            })
            break

pd.DataFrame(temp_arr)

,Feature,Type,Unique,Sample
0,loc,object,398,pune city
1,bt,object,12,sedan
2,tt,object,2,manual
3,images,object,37135,[{'img': 'https://stimg2.cardekho.com/images/u...
4,dvn,object,4128,bmw 5 series 520d modern line
5,oem,object,46,maruti
6,model,object,382,maruti baleno
7,variantName,object,3430,1.2 ti-vct titanium
8,city_x,object,617,hyderabad
9,utype,object,2,dealer


There is an additional issues here that needs to be addressed: 
Some of the columns have **more unique values** than are expected. For eg: `Turbo Charger` should either be a yes or a no or a NaN, beter are `5` unique values. The following are the columns suspected to have a higher than expected cardinality due to some quality issues: 
1. `Value Configuration`
2. `Turbo Charger`
3. `Gear Box`
4. `Drive Type`
5. `Steering Type`
6. `Front Brake Type`
7. `Rear Brake Type`
8. `Tyre Type`


In [16]:
# Make a list of such columns which seems to have a higher than expected cardinality
columns_unexpected_cardinality = [
    'Value Configuration',
    'Turbo Charger',
    'Gear Box',
    'Drive Type',
    'Steering Type',
    'Front Brake Type',
    'Rear Brake Type',
    'Tyre Type',
]

---

##### `Value Configuration`

In [17]:
cars_df[columns_unexpected_cardinality[0]].value_counts()

dohc                    23502
nan                      7831
sohc                     6200
undefined                 121
idsi                       66
dohc with vis              31
dohc with vgt              20
16 modules 48 cells        11
16-valve dohc layout        9
dohc with tis               9
vtec                        6
mpfi                        4
ohv / pushrod               3
Name: Value Configuration, dtype: int64

We have identified the following issues:
1. As we can see, the majority of it is `dohc`. There are a couple of variants of `dohc` like `dohc vis` but they are so few that it would be practical to replace all the columns containing any variant `dohc` to simply `dohc`
2. There are some `undefined` values which should be `NaN`
3. `mpfi` is a fuel injection system, so replace it with `NaN`
4. `vtec` engines can be `dohc` and `sohc` as well (`vtec` is an engine type developed by Honda in the 1960s). It would be best to replace this type by `NaN`, postpone handling it until null values handling (and there are only `6` rows, so it should be fine)

In [18]:
# Replace all the variants of `dohc` to simply `dohc`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vis', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with vgt', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('16-valve dohc layout', 'dohc')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('dohc with tis', 'dohc')

# Replace `undefined`, `mpfi`, `vtec` with `NaN`
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('undefined', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('mpfi', 'nan')
cars_df['Value Configuration'] = cars_df['Value Configuration'].str.replace('vtec', 'nan')

cars_df['Value Configuration'].value_counts()

dohc                   23571
nan                     7962
sohc                    6200
idsi                      66
16 modules 48 cells       11
ohv / pushrod              3
Name: Value Configuration, dtype: int64

---

##### `Turbo Charger`

In [19]:
cars_df[columns_unexpected_cardinality[1]].value_counts()

no       20247
yes      15262
nan       2176
twin       125
turbo        3
Name: Turbo Charger, dtype: int64

Well, cleaning this column should be quite straightforward.
1. `twin` and `turbo` should be replaced by `yes`

In [20]:
# Replace `twin` and `turbo` with `yes`
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('twin', 'yes')
cars_df['Turbo Charger'] = cars_df['Turbo Charger'].str.replace('turbo', 'yes')

cars_df['Turbo Charger'].value_counts()

no     20247
yes    15390
nan     2176
Name: Turbo Charger, dtype: int64

--- 

##### `Gear Box`

In [21]:
cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed                  22840
6 speed                   5876
5-speed                   1457
7 speed                   1256
6-speed                   1199
                         ...  
six speed manual             1
7g-dct                       1
amg speedshift dct 8g        1
amg 7-speed dct              1
ecvt                         1
Name: Gear Box, Length: 106, dtype: int64

Here the cardinality is high. And as is evident, several categories can be lumped together into a single one. Let's define a mapping to clearly indeicate which data should be lumped together.

In [22]:
gear_box_mapping = {}
gear_box_mapping['1 speed'] = [
	'single speed', 
	'single speed automatic', 
	'single speed reduction gear', 
	'single-speed transmission', 
]
gear_box_mapping['4 speed'] = [
	'4 speed', 
	'4-speed', 
]
gear_box_mapping['5 speed'] = [
	'5', 
	'5 - speed', 
	'5 gears', 
	'5 manual', 
	'5 speed', 
	'5 speed at+ paddle shifters', 
	'5 speed cvt', 
	'5 speed forward, 1 reverse', 
	'5 speed manual', 
	'5 speed manual transmission', 
	'5 speed+1(r)', 
	'5 speed,5 forward, 1 reverse', 
	'5-speed', 
	'5-speed`', 
	'five speed', 
	'five speed manual', 
	'five speed manual transmission', 
	'five speed manual transmission gearbox', 
]
gear_box_mapping['6 speed'] = [
	'6', 
	'6 speed', 
	'6 speed at', 
	'6 speed automatic', 
	'6 speed geartronic', 
	'6 speed imt', 
	'6 speed ivt', 
	'6 speed mt', 
	'6 speed with sequential shift', 
	'6-speed', 
	'6-speed at', 
	'6-speed automatic', 
	'6-speed autoshift', 
	'6-speed cvt', 
	'6-speed dct', 
	'6-speed imt', 
	'6-speed ivt', 
	'6-speed`', 
	'six speed  gearbox', 
	'six speed automatic gearbox', 
	'six speed automatic transmission', 
	'six speed geartronic, six speed automati', 
	'six speed manual', 
	'six speed manual transmission', 
	'six speed manual with paddle shifter', 
]
gear_box_mapping['7 speed'] = [
	'7 speed', 
	'7 speed 7g-dct', 
	'7 speed 9g-tronic automatic', 
	'7 speed cvt', 
	'7 speed dct', 
	'7 speed dsg', 
	'7 speed dual clutch transmission', 
	'7 speed s tronic', 
	'7-speed', 
	'7-speed dct', 
	'7-speed dsg', 
	'7-speed pdk', 
	'7-speed s tronic', 
	'7-speed s-tronic', 
	'7-speed steptronic', 
	'7-speed stronic', 
	'7g dct 7-speed dual clutch transmission', 
	'7g-dct', 
	'7g-tronic automatic transmission',
	'amg 7-speed dct',	
	'mercedes benz 7 speed automatic',
]
gear_box_mapping['8 speed'] = [
	'8', 
	'8 speed', 
	'8 speed cvt', 
	'8 speed multitronic', 
	'8 speed sport', 
	'8 speed tip tronic s', 
	'8 speed tiptronic', 
	'8-speed', 
	'8-speed automatic', 
	'8-speed automatic transmission', 
	'8-speed dct', 
	'8-speed steptronic', 
	'8-speed steptronic sport automatic transmission', 
	'8-speed tiptronic', 
	'8speed',
	'amg speedshift dct 8g', 
]
gear_box_mapping['9 speed'] = [
	'9 -speed', 
	'9 speed', 
	'9 speed tronic', 
	'9-speed', 
	'9-speed automatic', 
	'9g tronic', 
	'9g-tronic', 
	'9g-tronic automatic', 
	'amg speedshift 9g tct automatic',
]
gear_box_mapping['10 speed'] = [
	'10 speed', 
]
gear_box_mapping['cvt'] = [
'cvt', 
'e-cvt', 
'ecvt', 
]
gear_box_mapping['direct drive'] = [
'direct drive', 
]
gear_box_mapping['fully automatic'] = [
	'automatic transmission', 
	'fully automatic',
]
gear_box_mapping['nan'] = [
'nan',
'ags', 
'imt', 
'ivt', 
]

mapping_dict = {v: k for k, lst in gear_box_mapping.items() for v in lst}
cars_df['Gear Box'] = cars_df['Gear Box'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[2]].value_counts()

5 speed            25203
6 speed             7459
7 speed             1842
8 speed             1150
4 speed              760
cvt                  474
nan                  471
9 speed              399
fully automatic       20
1 speed               15
direct drive          11
10 speed               9
Name: Gear Box, dtype: int64

--- 

##### `Drive Type`

In [23]:
cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd                                         27456
nan                                          4496
rwd                                          2248
awd                                          1082
2wd                                           648
4wd                                           570
2 wd                                          369
4x2                                           297
4x4                                           229
front wheel drive                             176
two wheel drive                                98
all wheel drive                                32
rear wheel drive with esp                      29
two whhel drive                                26
permanent all-wheel drive quattro              21
rwd(with mtt)                                  14
rear-wheel drive with esp                       7
4 wd                                            7
all-wheel drive with electronic traction        5
four whell drive                                2


We will once again take the same approach and define a mapping since many categories are redundant here.

In [24]:
drive_type_mapping = {}
drive_type_mapping['fwd'] = ['fwd', 'front wheel drive']
drive_type_mapping['2wd'] = ['2wd', 'two wheel drive', '2 wd', 'two whhel drive']
drive_type_mapping['rwd'] = ['rwd', 'rear wheel drive with esp', 'rear-wheel drive with esp', 'rwd(with mtt)']
drive_type_mapping['awd'] = ['awd', 'all wheel drive', 'all-wheel drive with electronic traction', 'permanent all-wheel drive quattro']
drive_type_mapping['4wd'] = ['4wd', '4 wd', '4x4', 'four whell drive']
drive_type_mapping['nan'] = ['nan', '3']

mapping_dict = {v: k for k, lst in drive_type_mapping.items() for v in lst}
cars_df['Drive Type'] = cars_df['Drive Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[3]].value_counts()

fwd    27632
nan     4497
rwd     2298
2wd     1141
awd     1140
4wd      808
4x2      297
Name: Drive Type, dtype: int64

---

##### `Steering Type`

In [25]:
# Steering Type
cars_df[columns_unexpected_cardinality[4]].value_counts()

power         31920
electric       3895
nan             808
manual          652
electronic      344
electrical      138
epas             49
hydraulic         5
mt                1
motor             1
Name: Steering Type, dtype: int64

There are 2 kinds of steering types (broadly) - 
1. Power (electric)
2. Manual (hydraulic, etc.)

In [26]:
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electrical', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electric', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('electronic', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('epas', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('mt', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('motor', 'power')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')
cars_df['Steering Type'] = cars_df['Steering Type'].str.replace('hydraulic', 'manual')

cars_df[columns_unexpected_cardinality[4]].value_counts()

power     36348
nan         808
manual      657
Name: Steering Type, dtype: int64

---

##### `Front Brake Type` and `Rear Brake Type`

In [27]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc                                        22642
ventilated disc                             13178
ventilated discs                              400
solid disc                                    398
nan                                           327
disc & caliper type                           296
disk                                          205
ventilated disk                                78
drum                                           67
multilateral disc                              42
vantilated disc                                39
disc, 236 mm                                   22
ventlated disc                                 17
vacuum assisted hydraulic dual circuit w       12
ventillated disc                               11
disc & drum                                    10
disc,internally ventilated                      9
electric parking brake                          8
264mm ventilated discs                          8
caliper ventilated disc                         6


In [28]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum                                        29260
disc                                         5272
ventilated disc                               757
solid disc                                    612
nan                                           326
self-adjusting drum                           322
discs                                         301
disc & caliper type                           296
ventilated discs                              178
leading-trailing drum                         134
leading & trailing drum                        75
ventilated drum                                46
drums                                          40
self adjusting drum                            33
disc & drum                                    32
self adjusting drums                           31
drums 180 mm                                   22
drum in discs                                  17
vacuum assisted hydraulic dual circuit w       12
262mm disc & drum combination                   8


In [29]:
brake_type_mapping = {}
brake_type_mapping['disc'] = [
	'disc',
	'260mm discs',
	'disc brakes',
	'disc, 236 mm',
	'discs',
	'disk',
	'multilateral disc',
	'solid disc',
	'electric parking brake',
	'abs',
]
brake_type_mapping['ventilated disc'] = [
	'264mm ventilated discs',
	'booster assisted ventilated disc',
	'caliper ventilated disc',
	'disc brakes with inner cooling',
	'disc,internally ventilated',
	'vantilated disc',
	'ventilated & grooved steel discs',
	'ventilated disc',
	'ventilated disc with twin pot caliper',
	'ventilated discs',
	'ventilated disk',
	'ventillated disc',
	'ventillated discs',
	'ventlated disc',
	'ventilated drum in discs',
	'ventialte disc',
	'ventialted disc',
]
brake_type_mapping['carbon ceramic'] = [
	'carbon ceramic brakes',
	'carbon ceramic brakes.',
]
brake_type_mapping['disc & drum'] = [
	'disc & drum',
	'228.6 mm dia, drums on rear wheels',
	'262mm disc & drum combination',
	'drum in disc',
	'drum in discs',
]
brake_type_mapping['drum'] = [
	'drum',
	'203mm drums',
	'drum`',
	'drums',
	'drums 180 mm',
	'booster assisted drum',
	'drum brakes',
	'leading & trailing drum',
	'leading-trailing drum',
	'self adjusting drum',
	'self adjusting drums',
	'self-adjusting drum',
	'single piston sliding fist',
	'ventilated drum',
	'tandem master cylinder with servo assist',

]
brake_type_mapping['caliper'] = [
	'six piston claipers',
	'twin piston sliding fist caliper',
	'vacuum assisted hydraulic dual circuit w',
	'four piston calipers',
	'disc & caliper type',
]

In [30]:
mapping_dict = {v: k for k, lst in brake_type_mapping.items() for v in lst}
cars_df['Front Brake Type'] = cars_df['Front Brake Type'].replace(mapping_dict)
cars_df['Rear Brake Type'] = cars_df['Rear Brake Type'].replace(mapping_dict)

In [31]:
cars_df[columns_unexpected_cardinality[5]].value_counts()

disc               23334
ventilated disc    13759
nan                  327
caliper              312
drum                  70
disc & drum           10
carbon ceramic         1
Name: Front Brake Type, dtype: int64

In [32]:
cars_df[columns_unexpected_cardinality[6]].value_counts()

drum               29973
disc                6195
ventilated disc      944
nan                  326
caliper              310
disc & drum           64
carbon ceramic         1
Name: Rear Brake Type, dtype: int64

---

##### `Tyre Type`

In [33]:
cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless,radial               21770
tubeless                       5723
tubeless, radial               4994
tubeless tyres                 2408
radial                          606
radial, tubeless                389
tubeless radial tyres           301
radial, tubless                 296
nan                             257
tubeless tyres, radial          224
tubeless,radials                212
radial,tubeless                 158
tubless, radial                  92
runflat                          69
tubeless tyre                    50
tubeless, runflat                48
run-flat                         47
runflat tyres                    41
radial tubeless                  33
tubeless,runflat                 31
tubeless radial                  17
runflat tyre                     16
tubeless, radials                 7
radial with tube                  4
tubeless. runflat                 3
tubeless radial tyrees            3
tubeless tyres mud terrain        3
radial tyres                

In [34]:
tyre_type_mapping = {}
tyre_type_mapping['tubeless'] = [
	'tubeless tyres',
	'tubeless',
	'tubeless tyres mud terrain',
	'tubeless tyre',
]
tyre_type_mapping['tubeless radial'] = [
	'tubeless, radial',
	'tubeless,radial',
	'tubeless tyres, radial',
	'tubeless radial tyres',
	'radial, tubeless',
	'radial',
	'tubless, radial',
	'radial tubeless',
	'tubeless radial',
	'tubeless,radials',
	'tubeless radials',
	'radial,tubeless',
	'tubeless radial tyre',
	'radial, tubless',
	'tubless radial tyrees',
	'tubeless , radial',
	'tubeless, radials',
	'radial tyres',
]
tyre_type_mapping['runflat'] = [
	'runflat tyres',
	'runflat',
	'tubeless,runflat',
	'run-flat',
	'runflat tyre',
	'tubeless, runflat',
	'tubeless. runflat',
	'tubeless.runflat',
	'tubeless radial tyrees',
]
tyre_type_mapping['tube'] = [
	'radial with tube',
]

In [35]:
mapping_dict = {v: k for k, lst in tyre_type_mapping.items() for v in lst}
cars_df['Tyre Type'] = cars_df['Tyre Type'].replace(mapping_dict)

cars_df[columns_unexpected_cardinality[7]].value_counts()

tubeless radial    29108
tubeless            8184
runflat              260
nan                  257
tube                   4
Name: Tyre Type, dtype: int64

With this column, all of the features suspected of high cardinality are handled. We can move on to the other things that were pointed out earlier in the code.

---

---